# MCP Client Jupyter Notebook

This notebook demonstrates how to use the MCP Python SDK to connect to MCP servers and interact with their tools, resources, and prompts.

In [ ]:
!pip install mcp

In [2]:
import asyncio
import subprocess
import threading
import queue
from mcp import ClientSession
import nest_asyncio
from typing import List, Dict, Any, Optional, Tuple
import anyio
import mcp.types as types
from pprint import pprint
import os
from anthropic import Anthropic

# Enable nested event loops
nest_asyncio.apply()

class MCPNotebookClient:
    """MCP client that can manage multiple server connections"""
    def __init__(self):
        self.servers: Dict[str, Dict[str, Any]] = {}
        
    def start_servers(self, server_configs: List[Tuple[str, str, List[str]]]) -> None:
        """Start multiple MCP servers at once.
        
        Args:
            server_configs: List of tuples (server_id, command, args)
                Example: [
                    ("memory", "npx", ["-y", "@modelcontextprotocol/server-memory"]),
                    ("filesystem", "npx", ["-y", "@modelcontextprotocol/server-filesystem", "/path"])
                ]
        """
        for server_id, command, args in server_configs:
            if server_id in self.servers:
                print(f"Warning: Server {server_id} already running, skipping")
                continue
                
            self._setup_server(server_id, command, args)
            
    def _setup_server(self, server_id: str, command: str, args: List[str]) -> None:
        """Set up a single server."""
        full_command = [command] + args
        print(f"Starting server {server_id} with command: {' '.join(full_command)}")
        
        # Create server entry
        server = {
            'id': server_id,
            'process': None,
            'session': None,
            '_read_stream_writer': None,
            '_write_stream_reader': None,
            '_read_stream': None,
            '_write_stream': None,
            '_stdout_thread': None,
            '_stderr_thread': None,
            '_should_stop': False,
            '_stdout_queue': queue.Queue(),
            'tools': []
        }
        self.servers[server_id] = server
        
        try:
            # Start process
            server['process'] = subprocess.Popen(
                full_command,
                stdin=subprocess.PIPE,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                bufsize=1
            )
            
            # Set up streams
            read_stream_writer, read_stream = anyio.create_memory_object_stream(0)
            write_stream, write_stream_reader = anyio.create_memory_object_stream(0)
            server['_read_stream_writer'] = read_stream_writer
            server['_write_stream_reader'] = write_stream_reader
            server['_read_stream'] = read_stream
            server['_write_stream'] = write_stream
            
            # Start asyncio tasks
            loop = asyncio.get_event_loop()
            loop.create_task(self._process_stdout(server))
            loop.create_task(self._process_writes(server))
            
            # Start threads
            server['_stdout_thread'] = threading.Thread(
                target=self._stdout_reader, args=(server,))
            server['_stderr_thread'] = threading.Thread(
                target=self._stderr_reader, args=(server,))
            server['_stdout_thread'].daemon = True
            server['_stderr_thread'].daemon = True
            server['_stdout_thread'].start()
            server['_stderr_thread'].start()
            
            # Initialize session
            loop.run_until_complete(self._initialize_session(server))
            
        except Exception as e:
            print(f"Failed to start server {server_id}: {e}")
            self.stop_server(server_id)

    async def _process_stdout(self, server: Dict[str, Any]) -> None:
        """Process messages from stdout queue."""
        while not server['_should_stop']:
            try:
                line = server['_stdout_queue'].get(timeout=0.1)
                try:
                    message = types.JSONRPCMessage.model_validate_json(line)
                    await server['_read_stream_writer'].send(message)
                except Exception as exc:
                    print(f"Error processing message: {exc}")
                    await server['_read_stream_writer'].send(exc)
            except queue.Empty:
                await asyncio.sleep(0.1)

    async def _process_writes(self, server: Dict[str, Any]) -> None:
        """Process writes to stdin."""
        async for message in server['_write_stream_reader']:
            if server['_should_stop'] or not server['process']:
                break
            json_str = message.model_dump_json(by_alias=True, exclude_none=True)
            try:
                server['process'].stdin.write(json_str + "\n")
                server['process'].stdin.flush()
            except Exception as e:
                print(f"Error writing to process: {e}")

    def _stdout_reader(self, server: Dict[str, Any]) -> None:
        """Read from process stdout."""
        while not server['_should_stop'] and server['process']:
            line = server['process'].stdout.readline()
            if not line:
                break
            server['_stdout_queue'].put(line)

    def _stderr_reader(self, server: Dict[str, Any]) -> None:
        """Read from process stderr."""
        while not server['_should_stop'] and server['process']:
            line = server['process'].stderr.readline()
            if not line:
                break
            print(f"Server {server['id']} stderr: {line.strip()}")

    async def _initialize_session(self, server: Dict[str, Any]) -> Any:
        """Initialize the MCP session for a server."""
        try:
            server['session'] = ClientSession(server['_read_stream'], server['_write_stream'])
            await server['session'].__aenter__()
            result = await asyncio.wait_for(server['session'].initialize(), timeout=10)
            
            # Get tools after initialization
            tools_result = await server['session'].list_tools()
            server['tools'] = [{ 
                "name": f"{server['id']}_{tool.name}",  # Use underscore instead of dot
                "description": f"[{server['id']}] {tool.description}",
                "input_schema": tool.inputSchema
            } for tool in tools_result.tools]
            
            print(f"Connected to {server['id']}: {result.serverInfo.name} v{result.serverInfo.version}")
            return result
        except asyncio.TimeoutError:
            print(f"Initialization timed out for server {server['id']}")
            return None
        except Exception as e:
            print(f"Error during initialization of {server['id']}: {e}")
            return None

    def stop_server(self, server_id: str) -> None:
        """Stop a specific server."""
        if server_id not in self.servers:
            return
            
        server = self.servers[server_id]
        server['_should_stop'] = True
        
        if server['process']:
            self._cleanup_session(server)
            self._cleanup_process(server)
        
        del self.servers[server_id]

    def stop_all_servers(self) -> None:
        """Stop all running servers."""
        for server_id in list(self.servers.keys()):
            self.stop_server(server_id)

    def _cleanup_session(self, server: Dict[str, Any]) -> None:
        """Clean up the MCP session for a server."""
        loop = asyncio.get_event_loop()
        
        async def cleanup():
            if server['session']:
                try:
                    await server['session'].__aexit__(None, None, None)
                except Exception as e:
                    print(f"Error closing session for {server['id']}: {e}")
                server['session'] = None
                
            if server['_read_stream_writer']:
                await server['_read_stream_writer'].aclose()
            if server['_write_stream_reader']:
                await server['_write_stream_reader'].aclose()
                
        loop.run_until_complete(cleanup())

    def _cleanup_process(self, server: Dict[str, Any]) -> None:
        """Clean up the subprocess for a server."""
        if server['process']:
            try:
                server['process'].stdin.close()
                server['process'].terminate()
                server['process'].wait(timeout=5)
            except Exception as e:
                print(f"Error during process cleanup for {server['id']}: {e}")
            finally:
                server['process'] = None
                print(f"Server {server['id']} stopped")
                
    def call_tool(self, full_tool_name: str, arguments: Optional[dict] = None) -> Any:
        """Call a tool, parsing the server ID from the tool name."""
        parts = full_tool_name.split('_', 1)  # Split on underscore instead of dot
        if len(parts) != 2:
            raise ValueError(f"Invalid tool name format. Expected 'server_id_tool_name', got '{full_tool_name}'")
        
        server_id, tool_name = parts
        
        if server_id not in self.servers:
            raise RuntimeError(f"Server {server_id} not found")
            
        server = self.servers[server_id]
        if not server['session']:
            raise RuntimeError(f"Server {server_id} not connected")
            
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(server['session'].call_tool(tool_name, arguments or {}))

    def get_all_tools(self) -> List[Dict[str, Any]]:
        """Get a list of all tools from all connected servers."""
        all_tools = []
        for server in self.servers.values():
            all_tools.extend(server['tools'])
        return all_tools

In [3]:
class MCPChatInterface:
    """Chat interface that works with multiple MCP servers"""
    def __init__(self, client: MCPNotebookClient):
        self.client = client
        self.anthropic = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
        self.messages: List[Dict[str, Any]] = []
        
    def run_chat(self):
        """Run an interactive chat loop with Claude that can use tools from multiple servers."""
        print("Starting chat, type 'quit' to exit.")
        
        # Clear message history at start
        self.messages = []
        
        # Get available tools from all servers
        available_tools = self.client.get_all_tools()
        
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() == 'quit':
                print("Ending chat session.")
                break
                
            self.messages.append({
                "role": "user",
                "content": [{"type": "text", "text": user_input}]
            })
            
            try:
                while True:
                    
                    response = self.anthropic.messages.create(
                        model="claude-3-5-sonnet-20241022",
                        max_tokens=1024,
                        messages=self.messages,
                        tools=available_tools
                    )
                    
                    all_content = []
                    
                    for content in response.content:
                        if content.type == 'text':
                            if content.text:
                                print(f"\nClaude: {content.text}")
                                all_content.append({
                                    "type": "text", 
                                    "text": content.text
                                })
                        elif content.type == 'tool_use':
                            self._handle_tool_use(content, all_content)
                            # Start new content list for next tools/text
                            all_content = []
                            break  # Break the for loop when tool is used
                    else:
                        # This executes if no tool was used (no break)
                        if all_content:
                            self.messages.append({
                                "role": "assistant", 
                                "content": all_content
                            })
                        break  # Break the while loop
                    
                    # Add any remaining content as assistant message
                    if all_content:
                        self.messages.append({
                            "role": "assistant", 
                            "content": all_content
                        })
                        
            except Exception as e:
                print(f"\nError: {str(e)}")
                
    def _handle_tool_use(self, content, all_content):
        """Handle tool use request from Claude."""
        tool_name = content.name
        tool_args = content.input
        tool_id = content.id
        print(f"\nClaude wants to use tool: {tool_name}")
            
        try:
            result = self.client.call_tool(tool_name, tool_args)
            result_str = self._get_result_string(result)
            print(f"Tool result for {tool_name}: {result_str}")
            
            all_content.append({
                "type": "tool_use",
                "id": tool_id,        
                "name": tool_name,    
                "input": tool_args    
            })
            

            self.messages.append({
                "role": "assistant",
                "content": all_content
            })

            self.messages.append({
                "role": "user",
                "content": [{
                    "type": "tool_result",
                    "tool_use_id": tool_id,
                    "content": result_str
                }]
            })
            
        except Exception as e:
            error_msg = f"Error executing tool {tool_name}: {str(e)}"
            print(f"Tool error: {error_msg}")
            all_content.append({
                "type": "text",
                "text": f"Error using tool: {error_msg}"
            })

    def _get_result_string(self, result):
        """Convert tool result to string format."""
        if hasattr(result, 'content'):
            return '\n'.join(
                c.text for c in result.content 
                if hasattr(c, 'text')
            )
        return str(result)

In [5]:
client = MCPNotebookClient()
chat = MCPChatInterface(client)

In [6]:
# Cell 1: Start multiple servers at once
import os

# Get the directory where the notebook is running
current_dir = os.path.dirname(os.path.abspath('__file__'))
filesystem_test_dir = os.path.join(current_dir, 'filesystem-testing')

client.start_servers([
    ("memory", "npx", ["-y", "@modelcontextprotocol/server-memory"]),
    ("filesystem", "npx", ["-y", "@modelcontextprotocol/server-filesystem", filesystem_test_dir])
])

# Cell 2: Run the chat interface
chat.run_chat()

Starting server memory with command: npx -y @modelcontextprotocol/server-memory
Server memory stderr: Knowledge Graph MCP Server running on stdio
Connected to memory: memory-server v1.0.0
Starting server filesystem with command: npx -y @modelcontextprotocol/server-filesystem /Users/elie/code/mcp/mcp-client-notebook/filesystem-testing
Server filesystem stderr: Secure MCP Filesystem Server running on stdio
Server filesystem stderr: Allowed directories: [ '/Users/elie/code/mcp/mcp-client-notebook/filesystem-testing' ]
Connected to filesystem: secure-filesystem-server v0.2.0
Starting chat, type 'quit' to exit.

Claude: I'll help you check what's in the memory by reading the entire knowledge graph.

Claude wants to use tool: memory_read_graph
Tool result for memory_read_graph: {
  "entities": [
    {
      "type": "entity",
      "name": "Elie Schoppik",
      "entityType": "Person",
      "observations": [
        "Works at Anthropic"
      ]
    },
    {
      "type": "entity",
      "nam

In [92]:
client.stop_all_servers()

Error closing session for memory: Attempted to exit cancel scope in a different task than it was entered in
Server memory stopped
Error closing session for filesystem: Attempted to exit cancel scope in a different task than it was entered in
Server filesystem stopped
